In [1]:
import logging
from kubernetes import config
from testbed.client.manager import TestbedManager

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

config.load_kube_config()
manager = TestbedManager("testbed-dev", True)

In [2]:
deleted_count = manager.delete_all_testbeds()
print(f"Deleted {deleted_count} testbeds")

INFO:testbed.client.manager:Deleting all testbeds
INFO:testbed.client.manager:Deleted 1 testbeds


Deleted 1 testbeds


In [3]:
# For listing testbeds (summary view)
for testbed in manager.list_testbeds():
    print(f"Testbed ID: {testbed.testbed_id}")
    print(f"Instance ID: {testbed.instance_id}")
    print(f"Pod Phase: {testbed.status.pod_phase}")
    print(f"Testbed Ready: {testbed.status.testbed_ready}")
    print(f"Sidecar Ready: {testbed.status.sidecar_ready}")
    print("---")

    testbed_id = testbed.testbed_id

Testbed ID: sympy-sympy-21847-testbed-jgt78
Instance ID: sympy__sympy-21847
Pod Phase: Running
Testbed Ready: True
Sidecar Ready: True
---


In [4]:
instance_id = "sympy__sympy-21847"
response = manager.create_testbed(instance_id=instance_id)
testbed_id = response.testbed_id

print(f"Created testbed with id {testbed_id}")

INFO:testbed.client.manager:create_testbed(user: None, instance_id: sympy__sympy-21847)
INFO:testbed.client.manager:create_testbed(user: None, instance_id: sympy__sympy-21847, testbed_id: sympy-sympy-21847-testbed-s3umq) Job and Service created successfully in namespace testbed-dev.


Created testbed with id sympy-sympy-21847-testbed-s3umq


In [5]:
testbed = manager.wait_for_testbed_ready(testbed_id)

Testbed ID: sympy-sympy-21847-testbed-s3umq
Pod Phase: Running
External IP: 4.225.198.187
Testbed Container:
  Ready: True
  State: running
Sidecar Container:
  Ready: True
  State: running

Waiting for testbed to be ready... (Elapsed time: 26s)
Testbed sympy-sympy-21847-testbed-s3umq is ready and can be reached on http://4.225.198.187:8000!


In [6]:
client = manager.create_client(testbed_id, timeout=120)
print("Client created")
client.check_health()

INFO:testbed.client.manager:Health checking client
INFO:testbed.client.manager:Health check successful, client ready


Client created


True

In [18]:
client.execute_commands(["pwd"])

In [23]:
response = client.get_execution()

print(response.output)

/shared/commands.sh: 1: source: not found

CondaError: Run 'conda init' before 'conda activate'

On branch master
nothing to commit, working tree clean
commit d9b18c518d64d0ebe8e35a98c2fb519938b9b151
Merge: 8daad34079 2f5dd663dd
Author: Oscar Benjamin <oscar.j.benjamin@gmail.com>
Date:   Tue Aug 10 00:26:06 2021 +0100

    Merge pull request #21769 from redeboer/CG._latex
    
    Fixed rendering of Clebsch-Gordan (CG) class

/shared/commands.sh: 9: source: not found

CondaError: Run 'conda init' before 'conda activate'

Obtaining file:///testbed
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: sympy
    Found existing installation: sympy 1.9.dev0
    Uninstalling sympy-1.9.dev0:
      Successfully uninstalled sympy-1.9.dev0
  Running setup.py develop for sympy



In [20]:
from testbed.swebench.test_spec import make_env_setup_script_list
from testbed.swebench.utils import load_swebench_instance
from datasets import Dataset, load_dataset
from typing import cast, Optional
import json
dataset = cast(Dataset, load_dataset("princeton-nlp/SWE-bench_Lite", split="test"))

for instance in dataset:
    if instance["instance_id"] == instance_id:
        print(json.dumps(instance, indent=2))

instance = load_swebench_instance(instance_id=instance_id, name="princeton-nlp/SWE-bench_Lite")
setup_commands = make_env_setup_script_list(instance)

for command in setup_commands:
    print(command)


{
  "repo": "sympy/sympy",
  "instance_id": "sympy__sympy-21847",
  "base_commit": "d9b18c518d64d0ebe8e35a98c2fb519938b9b151",
  "patch": "diff --git a/sympy/polys/monomials.py b/sympy/polys/monomials.py\n--- a/sympy/polys/monomials.py\n+++ b/sympy/polys/monomials.py\n@@ -127,7 +127,7 @@ def itermonomials(variables, max_degrees, min_degrees=None):\n                 for variable in item:\n                     if variable != 1:\n                         powers[variable] += 1\n-                if max(powers.values()) >= min_degree:\n+                if sum(powers.values()) >= min_degree:\n                     monomials_list_comm.append(Mul(*item))\n             yield from set(monomials_list_comm)\n         else:\n@@ -139,7 +139,7 @@ def itermonomials(variables, max_degrees, min_degrees=None):\n                 for variable in item:\n                     if variable != 1:\n                         powers[variable] += 1\n-                if max(powers.values()) >= min_degree:\n+            

In [21]:
client.execute_commands(setup_commands)

In [10]:
from testbed.schema import Prediction
instance_id = "sympy__sympy-21847"

prediction = Prediction(run_id="testing", instance_id=instance_id, patch="")
result = client.run_evaluation(prediction)
print(result)

ERROR:testbed.client.client:Error during run_evaluation: 500 Server Error: INTERNAL SERVER ERROR for url: http://4.225.198.187:8000/run_evaluation


None


In [11]:
# manager.delete_testbed(testbed_id)